In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
pd.set_option('display.max_columns', 50)

In [3]:
sales_df = pd.read_csv(r'src/Pizza_sales.csv')
sales_df

,pizza_id,order_id,pizza_name_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name
0,1,1,hawaiian_m,1,1/1/2015,11:38:36,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza
1,2,2,classic_dlx_m,1,1/1/2015,11:57:40,16.00,16.00,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza
2,3,2,five_cheese_l,1,1/1/2015,11:57:40,18.50,18.50,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",The Five Cheese Pizza
3,4,2,ital_supr_l,1,1/1/2015,11:57:40,20.75,20.75,L,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza
4,5,2,mexicana_m,1,1/1/2015,11:57:40,16.00,16.00,M,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza
...,...,...,...,...,...,...,...,...,...,...,...,...
48615,48616,21348,ckn_alfredo_m,1,31-12-2015,21:23:10,16.75,16.75,M,Chicken,"Chicken, Red Onions, Red Peppers, Mushrooms, A...",The Chicken Alfredo Pizza
48616,48617,21348,four_cheese_l,1,31-12-2015,21:23:10,17.95,17.95,L,Veggie,"Ricotta Cheese, Gorgonzola Piccante Cheese, Mo...",The Four Cheese Pizza
48617,48618,21348,napolitana_s,1,31-12-2015,21:23:10,12.00,12.00,S,Classic,"Tomatoes, Anchovies, Green Olives, Red Onions,...",The Napolitana Pizza
48618,48619,21349,mexicana_l,1,31-12-2015,22:09:54,20.25,20.25,L,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza


# Preprocessing

#### Handling Missing values

In [4]:
sales_df.isna().sum()

pizza_id              0
order_id              0
pizza_name_id        16
quantity              0
order_date            0
order_time            0
unit_price            0
total_price           7
pizza_size            0
pizza_category       23
pizza_ingredients    13
pizza_name            7
dtype: int64

In [5]:
sales_df.nunique()

pizza_id             48620
order_id             21350
pizza_name_id           91
quantity                 4
order_date             358
order_time           16382
unit_price              25
total_price             56
pizza_size               5
pizza_category           4
pizza_ingredients       32
pizza_name              32
dtype: int64

In [6]:
ProductName = []
for i in sales_df[sales_df.pizza_name.isna()==True]['pizza_name_id']:

  i = i.replace('_n', '')
  name = 'The ' + i[:-2] +' Pizza'
  if '_' in name:
    name = name.replace('_', ' ')
  if "ital" in name:
    name = name.replace('ital', 'italian')

  ProductName.append(name.title())

ProductNameIndex =sales_df[sales_df.pizza_name.isna()==True]['pizza_name'].index

sales_df.loc[ProductNameIndex,'pizza_name'] = ProductName
sales_df.pizza_name.nunique()

32

In [7]:
TotalPrice = sales_df[sales_df.total_price.isna()==True]['quantity'] * sales_df[sales_df.total_price.isna()==True]['unit_price']
TotalPriceIndex = sales_df[sales_df.total_price.isna()==True].index

sales_df.loc[TotalPriceIndex,'total_price'] = TotalPrice
sales_df.total_price.isna().sum()

0

In [8]:
pizza_name = sales_df[sales_df.pizza_ingredients.isna()==True]['pizza_name']
PizzaIngredients = [sales_df[(sales_df['pizza_name']==name) & (sales_df.pizza_category.isna()==False)].pizza_ingredients.unique()[0] for name in pizza_name]
PizzaIngredientsIndex = sales_df[sales_df.pizza_ingredients.isna()==True].index

sales_df.loc[PizzaIngredientsIndex, 'pizza_ingredients'] = PizzaIngredients
sales_df.pizza_ingredients.isna().sum()

0

In [9]:
pizza_name = sales_df[sales_df.pizza_category.isna()==True]['pizza_name']
PizzaCategory = [sales_df[(sales_df['pizza_name']==name) & (sales_df.pizza_category.isna()==False)].pizza_category.unique()[0] for name in pizza_name]
PizzaCategoryIndex = sales_df[sales_df.pizza_category.isna()==True].index

sales_df.loc[PizzaCategoryIndex, 'pizza_category'] = PizzaCategory
sales_df.pizza_category.isna().sum()

0

In [10]:
Pizza_Size =sales_df[sales_df['pizza_name_id'].isna()==True]['pizza_size']
Pizza_Name = sales_df[sales_df['pizza_name_id'].isna()==True]['pizza_name']
PizzaNameID = [sales_df[(sales_df['pizza_name']==name) & (sales_df['pizza_size']==size)]['pizza_name_id'].mode()[0] for name, size in zip(Pizza_Name, Pizza_Size)]
PizzaNameIDIndex = sales_df[sales_df.pizza_name_id.isna()==True].index

sales_df.loc[PizzaNameIDIndex, 'pizza_name_id'] = PizzaNameID
sales_df.pizza_name_id.isna().sum()

0

In [11]:
sales_df['order_date'] = sales_df['order_date'].str.replace('/','-')
sales_df['order_date'] = pd.to_datetime(sales_df['order_date'], format ='%d-%m-%Y').dt.date


In [12]:
sales_df['week_number'] = pd.to_datetime(sales_df['order_date'], format ='%Y-%m-%d').dt.isocalendar().week

In [13]:
sales_df.isna().sum()

pizza_id             0
order_id             0
pizza_name_id        0
quantity             0
order_date           0
order_time           0
unit_price           0
total_price          0
pizza_size           0
pizza_category       0
pizza_ingredients    0
pizza_name           0
week_number          0
dtype: int64

### Converting Daily Dataset into Weekly Dataset

In [15]:
weekly_df =sales_df.groupby(['week_number','pizza_name', 'pizza_size'])['quantity'].sum().reset_index()
weekly_df

,week_number,pizza_name,pizza_size,quantity
0,1,The Barbecue Chicken Pizza,L,18
1,1,The Barbecue Chicken Pizza,M,15
2,1,The Barbecue Chicken Pizza,S,5
3,1,The Big Meat Pizza,S,21
4,1,The Brie Carre Pizza,S,3
...,...,...,...,...
4748,53,The Thai Chicken Pizza,M,4
4749,53,The Thai Chicken Pizza,S,4
4750,53,The Vegetables + Vegetables Pizza,L,6
4751,53,The Vegetables + Vegetables Pizza,M,4


In [16]:
def get_start_of_week(year, week_number):
    # Find the first day of the week
    start_of_week = datetime.strptime(f'{year}-W{week_number}-1', "%Y-W%U-%w")
    return start_of_week

weekly_df['sales_week_date'] = weekly_df['week_number'].apply(lambda week: get_start_of_week(2015, week))

weekly_df['year'] = weekly_df['sales_week_date'].dt.year
weekly_df['month'] = weekly_df['sales_week_date'].dt.month
weekly_df['day'] = weekly_df['sales_week_date'].dt.day

weekly_df.drop('sales_week_date', axis =1, inplace = True)

weekly_df

,week_number,pizza_name,pizza_size,quantity,year,month,day
0,1,The Barbecue Chicken Pizza,L,18,2015,1,5
1,1,The Barbecue Chicken Pizza,M,15,2015,1,5
2,1,The Barbecue Chicken Pizza,S,5,2015,1,5
3,1,The Big Meat Pizza,S,21,2015,1,5
4,1,The Brie Carre Pizza,S,3,2015,1,5
...,...,...,...,...,...,...,...
4748,53,The Thai Chicken Pizza,M,4,2016,1,4
4749,53,The Thai Chicken Pizza,S,4,2016,1,4
4750,53,The Vegetables + Vegetables Pizza,L,6,2016,1,4
4751,53,The Vegetables + Vegetables Pizza,M,4,2016,1,4


In [17]:
def to_get_unit_price(pizza_name, size):
  unit_price = sales_df[(sales_df['pizza_name'] == pizza_name)
                        & (sales_df['pizza_size']== size)]['unit_price'].mean()
  return unit_price
weekly_df['unit_price'] = weekly_df.apply(lambda row: to_get_unit_price(row['pizza_name'], row['pizza_size']), axis=1)
weekly_df.head()

,week_number,pizza_name,pizza_size,quantity,year,month,day,unit_price
0,1,The Barbecue Chicken Pizza,L,18,2015,1,5,20.75
1,1,The Barbecue Chicken Pizza,M,15,2015,1,5,16.75
2,1,The Barbecue Chicken Pizza,S,5,2015,1,5,12.75
3,1,The Big Meat Pizza,S,21,2015,1,5,12.00
4,1,The Brie Carre Pizza,S,3,2015,1,5,23.65


In [18]:
def to_get_pizza_category(pizza_name):
  pizza_category = sales_df[sales_df['pizza_name'] == pizza_name]['pizza_category'].mode()[0]
  return pizza_category
to_get_pizza_category('The Barbecue Chicken Pizza')
weekly_df['pizza_category'] = weekly_df['pizza_name'].apply(lambda name: to_get_pizza_category(name))
weekly_df.head()

,week_number,pizza_name,pizza_size,quantity,year,month,day,unit_price,pizza_category
0,1,The Barbecue Chicken Pizza,L,18,2015,1,5,20.75,Chicken
1,1,The Barbecue Chicken Pizza,M,15,2015,1,5,16.75,Chicken
2,1,The Barbecue Chicken Pizza,S,5,2015,1,5,12.75,Chicken
3,1,The Big Meat Pizza,S,21,2015,1,5,12.00,Classic
4,1,The Brie Carre Pizza,S,3,2015,1,5,23.65,Supreme


In [19]:
weekly_df

,week_number,pizza_name,pizza_size,quantity,year,month,day,unit_price,pizza_category
0,1,The Barbecue Chicken Pizza,L,18,2015,1,5,20.75,Chicken
1,1,The Barbecue Chicken Pizza,M,15,2015,1,5,16.75,Chicken
2,1,The Barbecue Chicken Pizza,S,5,2015,1,5,12.75,Chicken
3,1,The Big Meat Pizza,S,21,2015,1,5,12.00,Classic
4,1,The Brie Carre Pizza,S,3,2015,1,5,23.65,Supreme
...,...,...,...,...,...,...,...,...,...
4748,53,The Thai Chicken Pizza,M,4,2016,1,4,16.75,Chicken
4749,53,The Thai Chicken Pizza,S,4,2016,1,4,12.75,Chicken
4750,53,The Vegetables + Vegetables Pizza,L,6,2016,1,4,20.25,Veggie
4751,53,The Vegetables + Vegetables Pizza,M,4,2016,1,4,16.00,Veggie


In [20]:
weekly_df.to_csv(r'src/Weekly_Pizza_Sales.csv', index = False)